# How to implement a Variational Autoencoder (VAE)

A variational autoencoder observes data, infers a latent code for it and tries to reconstruct the data from that latent code. In contrast to regular autoencoders, the code of the VAE is **random**. That means that when presented with the same input, the VAE will produce a slightly different code each time. This makes its decoding process more robust, since it has to deal with noisy code.

Another way of looking at a VAE is as a training procedure for a probablistic model. The model is 
$$p(x) = \int p(z)p(x|z) dz$$
where $z$ is the latent code and $x$ is the data. During training we need to infer a posterior over $z$. In the case of a VAE this is done by neural network.

Assuming that the theory of VAEs has already been presented, we now dive straight into implementing them. If you need more background on VAEs, have a look at our [tutorial slides](https://github.com/philschulz/VITutorial/tree/master/modules) and the references therein.

# The Framework

For the purpose of this tutorial we are going to use [pytorch](https://pytorch.org) which is a scalable deep learning library for python. We are going to import and abbreviate it as "mx". We will use pytorch to define a computation graph. This is done using the [torch.nn library](https://pytorch.org/docs/stable/nn.html). When building the VAE, all the methods that you use should be prefixed with `torch.nn`.

In [ ]:
import logging
import pathlib

from typing import List, Tuple, Optional, Iterable
from abc import ABC

from collections import OrderedDict
import itertools
import math
import numpy as np

import torch
import torch.utils.data
from torch.autograd import Variable

from matplotlib import cm, pyplot as plt

Next, we specify a couple of constants that will help us to load the data.

In [ ]:
DEFAULT_LEARNING_RATE = 0.0003

TRAIN_SET = 'train'
VALID_SET = 'valid'
TEST_SET = 'test'
data_names = [TRAIN_SET, VALID_SET, TEST_SET]
test_set = [TEST_SET]
data_dir = pathlib.Path.cwd() / "binary_mnist"
USE_CUDA = False # Set to True to run on your gpu

if USE_CUDA:
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")


Finally, we set up basic logging facilities to print intermediate output.

In [ ]:
def plot_images(a, columns):
    width = height = int(math.sqrt(a.shape[1]))
    n = a.shape[0]
    rows = math.ceil(n / columns)
    cellsize = min(15 / columns, 5)
    if rows == 1 and columns > n:
        if n == 1:
            plot, axes = plt.subplots(rows, sharex="col", sharey="row", figsize=(cellsize * columns, cellsize * rows))
        else:
            plot, axes = plt.subplots(rows, n, sharex="col", sharey="row", figsize=(cellsize * columns, cellsize * rows))
    else:
        plot, axes = plt.subplots(rows, columns, sharex="col", sharey="row", figsize=(cellsize * columns, cellsize * rows))
    for r in range(rows):
        for c in range(columns):
            if (r * columns + c) >= n:
                break
            if rows == 1:
                if n == 1:
                    axes.imshow(np.reshape(a[(r * columns) + c,:], (width, height)), cmap=cm.Greys)
                else:
                    axes[c].imshow(np.reshape(a[(r * columns) + c,:], (width, height)), cmap=cm.Greys)
            else:
                axes[r][c].imshow(np.reshape(a[(r * columns) + c,:], (width, height)), cmap=cm.Greys)
    plt.show()

logging.basicConfig(level=logging.DEBUG, format="%(asctime)s [%(levelname)s -- %(name)s]: %(message)s", datefmt="%H:%M:%S")
logger = logging.getLogger("notebook")
mpl_logger = logging.getLogger("matplotlib")
mpl_logger.setLevel(logging.ERROR)

# The Data

Throughout the tutorial we will use the binarised MNIST data set consisting of images of handwritten digits (0-9). 
The binarisation was done by sampling each pixel from a Bernoulli distribution with the pixel's original intensity being the Bernoulli parameter (see [this paper](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.141.1680&rep=rep1&type=pdf) for details).Each pixel has been mapped to either 0 or 1, meaning that pixels are either fully on or off. We use this data set because it allows us to use a rather simple product of Bernoullis as a likelihood. We download the data into a folder called "binary_mnist". This may take some time.

In [ ]:
data_dir.mkdir(parents=True, exist_ok=True)
for data_set in data_names:
    file_name = "binary_mnist.{}".format(data_set)
    goal = data_dir / file_name
    if goal.exists():
        logger.info("Data file {} exists".format(file_name))
    else:
        logger.info("Downloading {}".format(file_name))
        link = "http://www.cs.toronto.edu/~larocheh/public/datasets/binarized_mnist/binarized_mnist_{}.amat".format(
            data_set)
        urllib.request.urlretrieve(link, goal)
        logger.info("Finished")

Alright, now we have the data on disk. We will load the training and test data later. Right now we just load the validation set to see what the data looks like. We'll grab 5 random numbers and visualise them (you can run this box several times to get 5 different digits each time). That the digits look a bit rough results from their pixels being binarised.

In [ ]:
file_name = data_dir / f"binary_mnist.{VALID_SET}"
logger.info(f"Reading {file_name} into memory")
valid_set = np.genfromtxt(file_name)

In [ ]:
n = 5
random_indices = np.random.randint(valid_set.shape[0], size=n)
random_pictures = valid_set[random_indices, :]
plot_images(random_pictures, 5)


After having acquainted ourselves with the data we can now proceed to build our VAE.

# Diagonal Gaussian VAE

The most basic VAE model is one where we assume that the latent variable is multiviariate Gaussian. We fix the prior to be standard normal. During inference, we use a multivariate Gaussian variational distribution with diagonal covariance matrix. This means that we are only modelling variance but not covariance (in fact, a k-dimensional Guassian with diagonal covariance has the same density as a product of k independent univariate Gaussians). Geometrically, this variational distribution can only account for axis-aligned elliptical densities. It is thus rather limited in its modelling capabilities. Still, because it uses a neural network under the hood, it is very expressive. 

In this tutorial, we will model the mist binarised digit data set. Each image is encoded as a 784-dimensional vector. We will model each of these vectors as a product of 784 Bernoullis (of course, there are better models but we want to keep it simple). Our likelihood is thus a product of independent Bernoullis. The resulting model is formally specified as 

\begin{align}z \sim \mathcal{N}(0,I) && x_i|z \sim Bernoulli(\text{NN}_{\theta}(z))~~~ i \in \{1,2,\ldots, 784\} \ .\end{align}

The variational approximation is given by $$q(z|x) = \mathcal{N}(\text{NN}_{\lambda}(x), \text{NN}_{\lambda}(x)).$$

Notice that both the Bernoulli likelihood and the Gaussian variational distribution use NNs to compute their parameters. The parameters of the NNs, however, are different ($\theta$ and $\lambda$, respectively).

## Implementation

We will spread our implementation across 3 classes. This design choice is motivated by the desire to make our models as modular as possible. This will later allow us to mix and match different likelihoods and variational distributions.

* **Generator**: This class defines our likelihood. Given a latent value, it produces a data sample or assigns a density to an existing data point.
* **InferenceNetwork**: This neural network computes the parameters of the variational approximation from a data point.
* **VAE**: This is the variational autoencoder. It combines a Generator and an InferenceNetwork and trains them jointly. Once trained, it can generate random data points or try to reproduce data presented to it.

Below we have specified these classes abstractly. Make sure you understand what each method is supposed to be doing.

In [ ]:
class Generator(torch.nn.Sequential):
    """
    Generator network.

    :param data_dims: Dimensionality of the generated data.
    :param layer_sizes: Size of each hidden layer in the network.
    :param act_type: The activation after each hidden layer.
    """

    def __init__(self, input_dims: int, data_dims: int, layer_sizes: List[int], act_type: str) -> None:
        super(Generator, self).__init__(OrderedDict([]))
        self.input_dims = input_dims
        self.data_dims = data_dims
        self.layer_sizes = layer_sizes
        self.act_type = act_type

        
class InferenceNetwork(torch.nn.Module):
    """
    A network to infer distributions over latent states.

    :param latent_variable_size: The dimensionality of the latent variable.
    :param layer_sizes: Size of each hidden layer in the network.
    :param act_type: The activation after each hidden layer.
    """

    def __init__(self, latent_variable_size: int, layer_sizes: List[int], act_type: torch.nn.Module) -> None:
        super(InferenceNetwork, self).__init__()
        self.latent_variable_size = latent_variable_size
        self.layer_sizes = layer_sizes
        self.act_type = act_type

    
class VAE(torch.nn.Module):
    """
    A variational autoencoding model (Kingma and Welling, 2014).

    :param generator: A generator network that specifies the likelihood of the model.
    :param inference_net: An inference network that specifies the distribution over latent values.
    """

    def __init__(self, generator: Generator, inference_net: InferenceNetwork) -> None:
        super(VAE, self).__init__()
        self.generator = generator
        self.inference_net = inference_net

    def forward(self, data: Variable) -> Variable:
        raise NotImplementedError()
        
    def generate_reconstructions(self, data: Variable, n: int) -> Variable:
        """
        Generate a number of reconstructions of input data points.

        :param data: The input data.
        :param n: Number of reconstructions per data point.
        :return: The reconstructed data.
        """
        raise NotImplementedError()

    def phantasize(self, n: int) -> Variable:
        """
        Generate data by randomly sampling from the prior.

        :param n: Number of sampled data points.
        :return: Randomly generated data points.
        """
        raise NotImplementedError()

## Exercise 1
Let us start by implementing the generator. This is pretty much a standard neural network. The main point of this exercise is to get you comfortable with pytorch. Complete all the TODOs below. Before starting, check pytorch's [fully connected layer](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear) and [activation functions](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity) (and [more activation functions](https://pytorch.org/docs/stable/nn.html#non-linear-activations-other)).

In [ ]:
class ProductOfBernoullisGenerator(Generator):
    """
    A generator that produces binary vectors whose entries are independent Bernoulli draws.

    :param data_dims: Dimensionality of the generated data.
    :param layer_sizes: Size of each hidden layer in the network.
    :param act_type: The activation after each hidden layer.
    """

    def __init__(self, *args, **kwargs) -> None:
        super(ProductOfBernoullisGenerator, self).__init__(*args, **kwargs)
        # TODO choose the correct output activation for a Bernoulli variable. This should be a class in
        # torch.nn
        output_activation = torch.nn.Sigmoid

        # TODO for each layer size in layer_sizes, introduce a layer that performs an affine transformation 
        # (implemented by torch.nn.Linear) followed by a non-linearity that is specified by output_activation.

        for i, size in enumerate([self.input_dims] + self.layer_sizes[:-1]):
            print(i, len(self.layer_sizes))
            self.add_module(f"gen_fc_{i}", torch.nn.Linear(size, self.layer_sizes[i]))
            self.add_module(f"gen_act_{i}", self.act_type())
        self.add_module(f"gen_fc_out", torch.nn.Linear(self.layer_sizes[-1], self.data_dims))
        self.add_module(f"gen_act_out", output_activation())


We now move on to the inference network. Recall that this network will return the parameters of a diagonal Gaussian. Thus, we need to return two vectors of the same size: a mean and a standard deviation vector. (Formally, the parameters of the Gaussian are the variances. However, from the derivation of the Gaussian reparametrisation we know that we
need the standard deviations to generate a Gaussian random variable $z$ as transformation of a standard Gaussian variable $\epsilon$.)

## Exercise 2.a
To train our VAE we need to compute the KL of the prior distribution from the variational approximation. Assuming that
the prior is standard normal the KL for the a diagonal Gaussian variational approximation is
$$ 0.5 * \left(\sum_i - \log(\sigma_i^2) - 1 + \sigma_i^2 + \mu_i^2 \right) \ . $$
We use this KL divergence in the get_kl_loss method below of the GaussianInferenceNetwork. (Losses in mxnet are defined using the [MakeLoss](https://mxnet.incubator.apache.org/api/python/symbol/symbol.html#mxnet.symbol.MakeLoss) symbol.) Please implement the function diagonal_gaussian_kl. Use the [sum](https://mxnet.incubator.apache.org/api/python/symbol/symbol.html#mxnet.symbol.sum) and [log](https://mxnet.incubator.apache.org/api/python/symbol/symbol.html#mxnet.symbol.log) symbols in the process.

In [ ]:
def diagonal_gaussian_kl(location: torch.Tensor, scale: torch.Tensor) -> torch.Tensor:
    """
    Computes the KL divergence of a standard normal distribution from a diagonal Gaussian.
    
    :param location: The mean of the diagonal Gaussian.
    :param scale: The standard deviations of the diagonal Gaussian.
    :return: The value of the Kl divergence.
    """
    #q = torch.distributions.normal.Normal(loc, scale)
    #p = torch.distributions.normal.Normal(torch.zeros_like(loc), torch.ones_like(scale))
    #kl = torch.distributions.kl.kl_divergence(q, p)
    kl = torch.sum(0.5 * ((-1 * scale.pow(2).log()) -1 + scale.pow(2) + location.pow(2)), dim=1)
    return kl


## Exercise 2.b
We can finally implement the Gaussian reparametrisation. Fill in the correct code to perform the sampling in the sample_latent_state method.

**Hint:** In this exercise you will need to draw a random Gaussian sample (see [here](https://mxnet.incubator.apache.org/api/python/symbol/symbol.html#mxnet.symbol.random_normal)). The operator requires a
shape whose first entry is the batch size. The batch size is not known to you during implementation, however.
You can leave it underspecified by choosing $0$ as a value. When you combine the sampling operator with another
operator mxnet will infer the correct the batch size for you.

In [ ]:
class GaussianInferenceNetwork(InferenceNetwork):
    """
    An inference network that predicts the parameters of a diagonal Gaussian and samples from that distribution.

    :param latent_variable_size: The dimensionality of the latent variable.
    :param layer_sizes: Size of each hidden layer in the network.
    :param act_type: The activation after each hidden layer.
    """

    def __init__(self, data_size, *args) -> None:
        super(GaussianInferenceNetwork, self).__init__(*args)
        self.data_size = data_size
        
        # We choose to share the first layer between the networks that compute the standard deviations
        # and means. This is a fairly standard design choice.
        self.shared = torch.nn.Sequential(OrderedDict({
                "inf_joint_fc": torch.nn.Linear(self.data_size, self.layer_sizes[0]),
                "inf_joint_act": self.act_type(),
            }))
        # location (live in R^d), for Gaussians this is also the mean of the random variable
        self.location = torch.nn.Sequential(
                OrderedDict(itertools.chain(*[
                    (
                        (f"inf_mean_fc_{i}", torch.nn.Linear(self.layer_sizes[i], size)),
                        (f"inf_mean_act_{i}", self.act_type()),
                    ) for i, size in enumerate(self.layer_sizes[1:])
                ]))
            )
        self.location.add_module("inf_mean_compute", torch.nn.Linear(self.layer_sizes[-1], self.latent_variable_size))

        # scale (live in strictly positive space), for Gaussians this is also the std 
        self.scale = torch.nn.Sequential(
                OrderedDict(itertools.chain(*[
                    (
                        (f"inf_variance_fc_{i}", torch.nn.Linear(self.layer_sizes[i], size)),
                        (f"inf_variance_act_{i}", self.act_type()),
                    ) for i, size in enumerate(self.layer_sizes[1:])
                ]))
            )
        self.scale.add_module("inf_variance_compute", torch.nn.Linear(self.layer_sizes[-1], self.latent_variable_size))

        # TODO: The standard deviation is constrained to be positive for non-degenerate Gaussians. Take the
        # prev_out symbol and transform it into a vector of positive reals.
        self.scale.add_module("inf_variance_softrelu", torch.nn.Softplus())

    def sample_latent_state(self, loc: Variable, scale: Variable, n: Optional[int] = 1) -> Variable:
        """
        Sample a value for the latent Gaussian variable.

        :param mean: The mean of the Gaussian.
        :param std: The standard deviation of the Gaussian.
        :param n: Number of samples to be produced.
        :return: A Gaussian sample.
        """
        eps = Variable(scale.data.new(n, scale.size()[1]).normal_())
        escale = scale.expand(n, -1)
        eloc = loc.expand(n, -1)

        return (eps * escale) + loc

    def forward(self, data):
        shared = self.shared(data)
        loc = self.location(shared)
        scale = self.scale(shared)
        
        z = self.sample_latent_state(loc, scale, scale.size()[0])
        
        return z, loc, scale
    

# Exercise 3

The only thing that is left to do is to implement VAE training. To train our VAE we will maximise the ELBO:
$$ \mathbb{E}\left[ \log p(x|z) \right] - \text{KL}(q(z|x)||p(z)) \ . $$
By default, MXNet's optimisers minimise losses instead of maximising objectives. We thus turn the ELBO into a loss by taking its negative. We already get the (sampled) cross entropy loss and (analytically computed) KL divergence as return values form calls to the Generator and inference network respectively. Your job is to combine them into one loss. In mxnet this can be done using the [Group symbol](https://mxnet.incubator.apache.org/api/python/symbol/symbol.html#mxnet.symbol.Group). If a list of losses is provided as an argument to this symbol, the optimiser will add up these losses.

**Note**: We could just as well have added up the values of the cross-entropy and the KL divergence and put the resulting loss into a MakeLoss symbol. The reason why we want to separate these two terms of the loss is that it allows us to output KL divergence. We will use this below when defining the KLMetric. Besides tracking the ELBO, which is our optimisation objective, we can also track to KL divergence to see how far the variational approximation moves away from the prior.

In [ ]:
class GaussianVAE(VAE):
    """
    A VAE with Gaussian latent variables. It assumes a standard normal prior on the latent values.

    :param generator: A generator network that specifies the likelihood of the model.
    :param inference_net: An inference network that specifies the Gaussian over latent values.
    """

    def forward(self, data):
        z, loc, scale = self.inference_net(data)
        y_pred = self.generator(z)
        
        return y_pred, loc, scale


    def generate_reconstructions(self, data: Variable, n: int) -> Variable:
        """
        Generate a number of reconstructions of input data points.

        :param data: The input data.
        :param n: Number of reconstructions per data point.
        :return: The reconstructed data.
        """
        _, loc, scale = self.inference_net(data)
        latent_state = self.inference_net.sample_latent_state(loc, scale, n)
        return self.generator(latent_state)


    def phantasize(self, n):
        """
        Generate data by randomly sampling from the prior.

        :param n: Number of sampled data points.
        :return: Randomly generated data points.
        """
        latent_state = Variable(torch.empty(n, self.inference_net.latent_variable_size).normal_())
        if USE_CUDA:
            latent_state = latent_state.cuda()
        return self.generator(latent_state)

# Constructing a VAE

We have now all the code for VAEs in place. Below, we have defined a factory method that makes it easier for you to play with different architectures.

In [ ]:
def construct_vae(latent_type: str,
                  likelihood: str,
                  generator_layer_sizes: List[int],
                  infer_layer_sizes: List[int],
                  latent_variable_size: int,
                  data_dims: int,
                  generator_act_type: torch.nn.Tanh,
                  infer_act_type: torch.nn.Tanh) -> VAE:
    """
    Construct a variational autoencoder

    :param latent_type: Distribution of latent variable.
    :param likelihood: Type of likelihood.
    :param generator_layer_sizes: Sizes of generator hidden layers.
    :param infer_layer_size: Sizes of inference network hidden layers.
    :param latent_variable_size: Size of the latent variable.
    :param data_dims: Dimensionality of the data.
    :param generator_act_type: Activation function for generator hidden layers.
    :param infer_act_type: Activation function for inference network hidden layers.
    :return: A variational autoencoder.
    """
    if issubclass(likelihood, Generator):
        generator = likelihood(input_dims=latent_variable_size, data_dims=data_dims, layer_sizes=generator_layer_sizes,
                                                 act_type=generator_act_type)
    else:
        raise Exception("{} is an invalid likelihood type. It should be a subclass of Generator".format(likelihood))

    if issubclass(latent_type, InferenceNetwork):
        inference_net = latent_type(data_dims, latent_variable_size,
                                                 infer_layer_sizes,
                                                 infer_act_type)
    else:
        raise Exception("{} is an invalid latent variable type. It should be a subclass of InferenceNetwork".format(latent_type))
        
    return GaussianVAE(generator=generator, inference_net=inference_net)


# Exercise 4

Your turn! Construct your own VAE below.

In [ ]:
vae = construct_vae(
    latent_type=GaussianInferenceNetwork,
    likelihood=ProductOfBernoullisGenerator,
    generator_layer_sizes=[200,500],
    infer_layer_sizes=[500,200],
    latent_variable_size=200,
    data_dims=784,
    generator_act_type=torch.nn.Tanh,
    infer_act_type=torch.nn.Tanh
)
if USE_CUDA:
    vae = vae.cuda()

To check what your VAE looks like we will visualise it.

In [ ]:
print(vae)

# Training a VAE

We are now set to train the VAE. In order to do so we have to define a data iterator and a module in pytorch. The data iterator takes care of batching the data while the module executes the computation graph and updates the model parameters. For the purpose of training the model, we only need the training data which we load from disk.

In [ ]:

mnist = {}
file_name = data_dir / f"binary_mnist.{TRAIN_SET}"
logging.info(f"Reading {file_name} into memory")
mnist[TRAIN_SET] = np.genfromtxt(file_name)

logging.info("%s contains %d data points", file_name, mnist[TRAIN_SET].shape[0])

Using that data we define the training set loader. At this point we also need to choose a batch_size that will then be used in training. Notice that we randomise the order in which the data points are presented in the training set. Notice that the data and labels are identical. This is because the VAE tries to reconstruct its input data and thus labels and data are the same.

In [ ]:
batch_size = 200

train_data = torch.Tensor(mnist[TRAIN_SET])
train_label = torch.Tensor(mnist[TRAIN_SET])
if USE_CUDA:
    train_data = train_data.cuda()
    train_label = train_label.cuda()

dataset = torch.utils.data.TensorDataset(train_data, train_label)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)


Training the vae (or any other network defined in mxnet) is most easily done using the [fit method](https://mxnet.incubator.apache.org/api/python/module/module.html#mxnet.module.BaseModule.fit). Here we choose to train our model for 20 epochs. Our optimiser will be adam. Training will take some time (approximately 2-5 minutes, depending on your machine). We are keeping track of the loss (the negative ELBO) to see how the model develops. Notice that this loss
is not the actual negative ELBO but a _doubly stochastic approximation_ to it (see [here](https://projecteuclid.org/download/pdf_1/euclid.aoms/1177729586) and [here](http://proceedings.mlr.press/v32/titsias14.pdf)). The two sources of stochasticity are the mini-batches (the ELBO is defined with respect to the entire data set) and the Gaussian reparametrisation (we approximate the integral over $ z $ through sampling). Both sources of stochasticity leave the approximation unbiased, however. 

In [ ]:
epochs = 30

bce_func = torch.nn.BCELoss(reduction="none")
optimizer = torch.optim.Adam(vae.parameters(), lr=DEFAULT_LEARNING_RATE)

for t in range(epochs):
    running_loss = div = 0.0
    for i, data in enumerate(trainloader, 0):
        x,y = data
        y_pred, loc, scale = vae(x)
        bce = bce_func(y_pred, y).sum(dim=1).mean()

#        # [batch_size]
        running_loss += bce.item()
        kl = diagonal_gaussian_kl(loc, scale).mean()
        div += kl.item()
        loss = bce + kl
    
        elbo = -running_loss - div
        vae.zero_grad()
    
        loss.backward()
        optimizer.step()
    
        # print statistics
        if i % 20 == 19:    # print every 20 mini-batches
            logger.info("[%5d, %5d] elbo: %.3f; divergence: %.3f", t + 1, i + 1, elbo / 20, div / 20)
            running_loss = 0.0
            div = 0.0
    torch.save(vae.state_dict(), f"vae-{t:04d}.torch")

# Simulating Data from the VAE

Now that we have trained the VAE, we can use it to produce data for us. We produce 10 random digits. The pixel intensities are the predicted Bernoulli parameters.

In [ ]:
num_samples = 10

samples = vae.phantasize(num_samples)
samples = samples.cpu().detach().numpy()
print(samples.shape)
plot_images(samples, columns=5)

Notice that the above samples were obtained by sampling from the standard normal prior. We can also check the reconstruction abilities of the VAE. To do this we sample a random number from the test set and generate 10 reconstructions of it.

In [ ]:
file_name = data_dir / f"binary_mnist.{TEST_SET}"
logging.info(f"Reading {file_name} into memory")
mnist[TEST_SET] = np.genfromtxt(file_name)

Here is what the sampled number looks like. (Rerun if you want a different number.)

In [ ]:
random_indices = np.random.randint(mnist[TEST_SET].shape[0], size=1)
random_pictures = mnist[TEST_SET][random_indices, :]
plot_images(random_pictures, 1)

And now we are going to see how the VAE reconstructs it.

In [ ]:
num_samples = 10

random_x = torch.Tensor(random_pictures)
if USE_CUDA:
    random_x = random_x.cuda()
y_pred = (vae.generate_reconstructions(random_x, num_samples)
            .cpu()
            .detach()
            .numpy()
         )
plot_images(y_pred, columns=5)

# Congrats!

You have completed your first VAE tutorial. From here on you can modify this code to play with your own VAE ideas. Want to you use different data? Or vary the neural net structure for the generator or inference networks? Or use a different likelihood model? The world of probablistic modeling is yours to explore.